# RAG แบบง่าย
เป็นแบบง่ายที่สุดเพื่อให้เห็นขั้นตอนการทำงานคร่าวๆใช้ Gemini ในการทำเก็บข้อมูลในหน่วยความจำ 

In [5]:
!uv add scikit-learn google-genai numpy

Resolved 121 packages in 4ms
Audited 117 packages in 0.05ms


In [2]:
import os
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from google import genai
from google.genai.types import EmbedContentConfig
if not os.environ.get("GEMINI_API_KEY"):
    raise ValueError("Please set the GEMINI_API_KEY environment variable")

client = genai.Client()


## Embedding
ตัวอย่างนี้ใช้ "text-embedding-004" ซึ่งเป็นโมเดลของ Gemini

In [3]:
def get_embedding(text):
    """Get text embedding from Gemini API"""
    response = client.models.embed_content(
        model="text-embedding-004",
        contents=[text],
        config=EmbedContentConfig(
            task_type="RETRIEVAL_QUERY" if len(text) < 100 else "RETRIEVAL_DOCUMENT",
            output_dimensionality=768,
        ),
    )
    return np.array(response.embeddings[0].values)

In [4]:
get_embedding("hello world")

array([ 2.34180450e-02, -1.59174330e-02, -3.49098900e-02, -2.42186300e-04,
       -9.88026000e-03, -6.20920400e-03,  5.34120600e-02,  1.00416710e-02,
        3.00759020e-02,  4.07208400e-02, -5.08474000e-02,  2.34877710e-03,
        3.53100100e-02, -1.13612480e-02, -2.41144450e-02, -2.73957890e-02,
       -9.12757500e-03,  2.26069820e-03, -1.13750026e-01,  1.39861500e-02,
        6.51229600e-03,  4.00020550e-03, -2.72912590e-02, -5.82575800e-02,
       -4.43566360e-03,  4.97838100e-03,  4.59926500e-03,  3.34949980e-02,
        1.68244900e-02,  3.63910460e-02, -3.65744460e-02,  4.69152740e-02,
        4.14002570e-05, -2.41220910e-02,  2.07875910e-04,  9.34617900e-03,
       -5.66121000e-02,  2.60704290e-02, -3.71312580e-03, -5.90219270e-02,
       -2.38868390e-02,  3.14986200e-02,  1.50431390e-02,  1.91629700e-02,
        6.02326130e-02, -2.38014800e-02,  2.37245100e-02, -4.10975100e-03,
        1.01875600e-02, -6.52980570e-03,  5.59614560e-02,  4.65294070e-03,
       -9.76778200e-02,  

In [6]:
def create_embeddings(data_json):
    """Create and return embeddings data from name and description - this is the time-consuming step"""
    print("Generating embeddings...")
    # เลือก name descript มาทำ embeddings เพิ่มข้อมูลอื่นๆได้เท่าที่ต้องการ
    texts = [f"{r['name']}: {r['description']}" for r in data_json]
    
    # Generate all embeddings
    embeddings = []
    for i, text in enumerate(texts):
        print(f"  Embedding {i+1}/{len(texts)}: {data_json[i]['name']}")
        embedding = get_embedding(text)
        embeddings.append(embedding)
    
    print("Embedding generation complete!\n")
    return embeddings

## similarity Search
ค้นหาความคล้ายโดยเปรียบเทียบค่า embeddings ของเอกสารกับ query

In [7]:
def search_similarity(query, data_json, embeddings, top_k=5):
    """Search based on query and pre-generated embeddings"""
    print(f"Generating query embedding for: '{query}'")    
    # query มาทำ embedding
    query_embedding = get_embedding(query)    
    print("Calculating similarities...")
    # คำนวน similarities ระหว่าง query กับ document ทั้งหมด
    similarities = []
    for i, doc_embedding in enumerate(embeddings):
        similarity = cosine_similarity([query_embedding], [doc_embedding])[0][0]
        similarities.append({
            "item": data_json[i],
            "similarity": similarity
        })
    
    # เรียง similarity จากมากไปน้อย
    sorted_results = sorted(similarities, key=lambda x: x["similarity"], reverse=True)
    # ส่งออกไป k อันดับแรก
    return sorted_results[:top_k]

In [8]:
data_json = []
with open('./data2.json') as f:
    data_json = json.load(f)
data_embeddings = create_embeddings(data_json)

Generating embeddings...
  Embedding 1/9: คู่มือใช้งานแอปธนาคาร
  Embedding 2/9: คำเตือนภัยหลอกลวง
  Embedding 3/9: นโยบายความปลอดภัยบัญชี
  Embedding 4/9: สร้างประวัติเครดิตที่ดี
  Embedding 5/9: ระวัง SMS/ลิงก์ปลอม
  Embedding 6/9: การใช้งานผ่านเวป
  Embedding 7/9: การใช้งาน ATM
  Embedding 8/9: การใช้อีเมลให้ปลอดภัย
  Embedding 9/9: ทำอย่างไรเพื่อหลีกเลี่ยง Spyware
Embedding generation complete!



ค้นหาข้อมูลที่ใกล้เคียงกับ query

In [9]:
query = "อาหารไทย"
results = search_similarity(query,data_json,data_embeddings,4)

print(f"\nTop {len(results)} matching:")
for i, result in enumerate(results):
    item = result["item"]
    similarity = result["similarity"]
    print(f"\n{i+1}. {item['name']} (Relevance: {similarity:.4f})")
    print(f"   {item['description']}")

Generating query embedding for: 'อาหารไทย'
Calculating similarities...

Top 4 matching:

1. สร้างประวัติเครดิตที่ดี (Relevance: 0.6071)
   รักษาประวัติการชำระหนี้ให้ดีอยู่เสมอ เพื่อให้ธนาคารเชื่อมั่นในความสามารถในการชำระหนี้ของคุณ 

2. ระวัง SMS/ลิงก์ปลอม (Relevance: 0.6018)
   ไม่กดลิงก์หรือดาวน์โหลดแอปพลิเคชันจาก SMS ที่แอบอ้างเป็นธนาคาร เพราะอาจเป็นไวรัสที่ขโมยข้อมูล 

3. คู่มือใช้งานแอปธนาคาร (Relevance: 0.5988)
   การโอนเงินผ่านมือถือให้ไปที่เมนูโอนเงิน เลือกบัญชีผู้รับ ตรวจสอบชื่อและหมายเลขบัญชีให้ถูกต้อง 
ค่าธรรมเนียมการโอนภายในธนาคารเดียวกันไม่มีค่าใช้จ่าย การโอนไปต่างธนาคารคิดค่าธรรมเนียมตามเงื่อนไข 
หากโอนผิดบัญชีให้ติดต่อคอลเซ็นเตอร์ทันที เพื่อดำเนินการระงับและติดตามยอด

4. คำเตือนภัยหลอกลวง (Relevance: 0.5958)
   อย่าคลิกลิงก์ที่ส่งมาทาง SMS หรือ LINE จากผู้ไม่รู้จัก อย่าให้รหัส OTP กับใคร 
มิจฉาชีพมักอ้างเป็นเจ้าหน้าที่ธนาคารหรือหน่วยงานรัฐ เพื่อลวงให้โอนเงินหรือติดตั้งแอปอันตราย 
หากสงสัยให้โทรติดต่อหน่วยงานผ่านเบอร์ทางการด้วยตนเอง


## Generate
นำ context ที่ได้จาก search_similarity และ query ไปสร้างเป็น prompt เพื่อให้ LLM อธิบายเป็นภาษาที่เข้าใจได้ พูดง่ายๆคือให้มันอ่านผลการค้นหามาตอบ

In [10]:
def generate_response(query, context):
    """Generate a response using Gemini based on the query and retrieved context"""
    prompt = f"""
You are a helpful assistant .
Use the provided restaurant information to answer the user's query.
If the query asks for something not in the provided information, politely indicate 
that you don't have that specific information but suggest the closest alternatives.

USER QUERY: {query}

INFORMATION:
{context}

Please provide a helpful response that directly answers the user's query based on information above.
Include useful information
"""
    
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=prompt,
    )
    
    return response.text


โหลดข้อมูลเพื่อสร้างฐานข้อมูลใหม่

In [11]:
data_json = []
with open('./data2.json') as f:
    data_json = json.load(f)
data_embeddings = create_embeddings(data_json)

Generating embeddings...
  Embedding 1/9: คู่มือใช้งานแอปธนาคาร
  Embedding 2/9: คำเตือนภัยหลอกลวง
  Embedding 3/9: นโยบายความปลอดภัยบัญชี
  Embedding 4/9: สร้างประวัติเครดิตที่ดี
  Embedding 5/9: ระวัง SMS/ลิงก์ปลอม
  Embedding 6/9: การใช้งานผ่านเวป
  Embedding 7/9: การใช้งาน ATM
  Embedding 8/9: การใช้อีเมลให้ปลอดภัย
  Embedding 9/9: ทำอย่างไรเพื่อหลีกเลี่ยง Spyware
Embedding generation complete!



## RAG
ค้นหาแบบ similarity แล้วให้ LLM อ่านมาตอบ

In [12]:
# user_query = "ร้านอาหารอะไรมีเมนูอาหารไทย"
user_query = "ถ้าได้ SMS น่าสงสัยควรทำอย่างไร"
similarity_results = search_similarity(user_query,data_json,data_embeddings,3)
response = generate_response(user_query, similarity_results)
print("GEMINI RESPONSE: ", response)


Generating query embedding for: 'ถ้าได้ SMS น่าสงสัยควรทำอย่างไร'
Calculating similarities...
GEMINI RESPONSE:  หากได้รับ SMS น่าสงสัย ควรปฏิบัติดังนี้:

*   **ไม่ควรกดลิงก์** หรือดาวน์โหลดแอปพลิเคชันจาก SMS ที่แอบอ้างเป็นธนาคารหรือหน่วยงานต่าง ๆ เพราะอาจเป็นไวรัสที่ขโมยข้อมูลของคุณได้
*   **อย่าคลิกลิงก์** ที่ส่งมาทาง SMS หรือ LINE จากผู้ไม่รู้จัก
*   **อย่าให้รหัส OTP (One-Time Password) กับใคร** โดยเด็ดขาด
*   หากสงสัยว่า SMS มาจากหน่วยงานใด ให้โทรติดต่อหน่วยงานนั้นผ่าน**เบอร์โทรศัพท์ทางการด้วยตนเอง** เพื่อตรวจสอบข้อมูล ไม่ควรใช้เบอร์ที่ระบุใน SMS นั้นๆ

มิจฉาชีพมักอ้างเป็นเจ้าหน้าที่ธนาคารหรือหน่วยงานรัฐ เพื่อลวงให้โอนเงินหรือติดตั้งแอปอันตราย ดังนั้นควรระมัดระวังเป็นพิเศษครับ
